---
# VISUAL ANALYTICS OF TRAFFIC SIMULATION DATA
---

Code developed to: TrafficSim-Vis

   This code was developed to process data from MATSim outputs. And its main purpose is to achieve the visualization of the simulations through a single-page application.                                    
                                                                         
   To run it you only need:                                              
    - configure the coordenates that you want. Example: 'epsg:32717'  in          crs_source = Proj(init='epsg:32717')                                                                              

**DATA PREPROCESSING**



You need to load:

 1. *output_network.xml.gz*
 2. *output_events.xml.gz*

files generated by MATSim into your simulation.


In [ ]:
pip install matsim-tools

In [ ]:
# libraries
import json
import gzip
import math
import shutil
import matsim
import pandas as pd
from pyproj import Transformer
import xml.etree.ElementTree as ET
from pyproj import Proj, transform
from collections import defaultdict

---
# Network: nodes and links
---



In [ ]:
net = matsim.read_network('output_network.xml.gz')
# net.nodes
# net.links


Set coordinates:
Convert the coordinates of MATSim files to a global coordinate system such as EPSG:4326 (longitude, latitude) for the correct use of the tool.

(If you don't know the coordinates, you can enter the file "output_network.xml" and in the first lines you will find it as "EPSG:XXXX")

In [ ]:
#crs_source = Proj(init='epsg:25832') #kelheim
#crs_source = Proj(init='epsg:32719') #santiago

# cordenates source
crs_source = Proj(init='epsg:32717') #cuenca

# cordenates target (global coordinate)
crs_target = Proj(init='epsg:4326')

def transform_coordinates(row):
    x, y = row['x'], row['y']
    lon, lat = transform(crs_source, crs_target, x, y)
    return lon, lat

net.nodes['longitude'], net.nodes['latitude'] = zip(*net.nodes.apply(transform_coordinates, axis=1))


/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-5-14a299c774f4>:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(crs_source, crs_target, x, y)


In [ ]:
net.nodes

,x,y,node_id,longitude,latitude
0,724210.088347,9.680395e+06,1000916573,-78.982950,-2.889745
1,724280.914322,9.680457e+06,1000916592,-78.982314,-2.889177
2,723583.699994,9.681419e+06,1000916595,-78.988599,-2.880489
3,724798.101114,9.680901e+06,1000916603,-78.977670,-2.885160
4,723582.152213,9.681444e+06,1000916606,-78.988613,-2.880271
...,...,...,...,...,...
13734,723800.073160,9.681667e+06,999956778,-78.986657,-2.878250
13735,721031.218150,9.681493e+06,999956783,-79.011553,-2.879867
13736,722295.121636,9.681798e+06,999956784,-79.000192,-2.877086
13737,723186.456439,9.681356e+06,999956787,-78.992170,-2.881066


Here will be generated the parsed network file necessary for the simulation (download it).

In [ ]:
feature_collection = {
    "type": "FeatureCollection",
    "features": []
}

node_mapping = {(node["node_id"]): {"longitude": (node["longitude"]), "latitude": (node["latitude"])} for _, node in net.nodes.iterrows()}

for _, node in net.nodes.iterrows():
      feature = {
        "type": "Feature",
        "properties": {
            "id": (node["node_id"])
        },
        "geometry": {
            "type": "Point",
            "coordinates": [(node["longitude"]), (node["latitude"])]
        }
      }
      feature_collection["features"].append(feature)


for _, link in net.links.iterrows():

    from_coordinates = [node_mapping[(link["from_node"])]["longitude"], node_mapping[(link["from_node"])]["latitude"]]
    to_coordinates = [node_mapping[(link["to_node"])]["longitude"], node_mapping[(link["to_node"])]["latitude"]]

    feature = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [from_coordinates, to_coordinates]
        },
        "properties": {
            "id": (link["link_id"]),
            "from": (link["from_node"]),
            "to": (link["to_node"]),
            "length": float(link["length"]),
            "freespeed": float(link["freespeed"]),
            "capacity": float(link["capacity"]),
            "permlanes": float(link["permlanes"]),
            "oneway": int(link["oneway"]),
            "modes": (link["modes"])
        }
    }
    feature_collection["features"].append(feature)


with open('parsing_network.json', 'w') as json_file:
    json.dump(feature_collection, json_file, separators=(',', ':'))
print(f"Network file generated succesfull.")

Network file created succesfull.


------
# EVENTS

------

In [ ]:
events = matsim.event_reader('output_events.xml.gz', types=('entered link'))
df_events = pd.DataFrame(events)

In [ ]:
row_del_events = ['type']
df_events = df_events.drop(columns=row_del_events)

row_del_links  = ['to_node','length','freespeed','capacity','permlanes','oneway','modes']
net.links = net.links.drop(columns=row_del_links)

row_del_nodes  = ['x','y']
net.nodes = net.nodes.drop(columns=row_del_nodes)


In [ ]:
df_events = df_events.rename(columns={'link': 'link_id'})
df_events = df_events.merge(net.links, on='link_id', how='left')
df_events = df_events.rename(columns={'from_node': 'node_id'})
df_events = df_events.merge(net.nodes, on='node_id', how='left')

In [ ]:
del net.nodes
del net.links
df_events

,time,type,link_id,vehicle,node_id,longitude,latitude
0,16886.0,entered link,5329917980002r,1810,5171643469,-78.957406,-2.864589
1,16930.0,entered link,899646210008r,1810,5171643472,-78.957862,-2.863026
2,16953.0,entered link,899646210006r,1810,1042829842,-78.958559,-2.863502
3,17060.0,entered link,757597420019f,1810,894375205,-78.961924,-2.865615
4,17137.0,entered link,757597420025f,1810,894375196,-78.961936,-2.868423
...,...,...,...,...,...,...,...
56706,71134.0,entered link,4089042780025f,124,4108077033,-78.998733,-2.868528
56707,71134.0,entered link,5589089650002f,186,964822903,-78.966635,-2.865601
56708,71162.0,entered link,757597420000f,186,964818411,-78.965866,-2.864105
56709,71215.0,entered link,4089042780031f,124,4121435643,-78.999729,-2.866492


Here will be generated the parsed event(s) file necessary for the simulation (download it).

In [ ]:
batchesEvent=1000000
total_parts = math.ceil(len(df_events) / batchesEvent)

for part in range(total_parts):
    start_index = part * batchesEvent
    end_index = min((part + 1) * batchesEvent, len(df_events))

    df_subset = df_events.iloc[start_index:end_index]

    json_data = []

    for vehicle_id, group in df_subset.groupby('vehicle'):
        path_data = {
            "vehicle": str(vehicle_id),
            "path": group[['longitude', 'latitude']].values.tolist(),
            "timestamps": group['time'].tolist()
        }
        json_data.append(path_data)

    with open(f'parser_events_part_{part + 1}.json', 'w') as json_file:
        json.dump(json_data, json_file, separators=(',', ':'))

    print(f"Part {part + 1} from Event file generated succesfull.")


Part 1 from event json file generated succesfull.
